In [1]:
from datasets import load_from_disk, load_metric, load_dataset
from transformers import Wav2Vec2BertProcessor, Wav2Vec2BertForCTC, BatchFeature
from transformers import Wav2Vec2CTCTokenizer, SeamlessM4TFeatureExtractor, Wav2Vec2BertForCTC, Wav2Vec2ProcessorWithLM, Wav2Vec2BertProcessor
from transformers import AutoModelForCTC, Wav2Vec2Processor
from torch.utils.data.dataloader import DataLoader
import torch
from tqdm import tqdm
from typing import Any, Dict, List, Optional, Union
from dataclasses import dataclass, field

import os
from datasets import load_from_disk
import numpy as np

In [10]:
# dataset_for_wav2vec2=load_from_disk("/teamspace/studios/this_studio/datasets/ivritai")
# dataset = dataset_for_wav2vec2['test']
# shuffled_dataset = dataset.shuffle(seed=np.random.randint(1000))
# dataset = shuffled_dataset.select(range(1000))

dataset=load_dataset("mozilla-foundation/common_voice_17_0", 'he', split="train")
transcription_key_in_dictionary= 'sentence'
shuffled_dataset = dataset.shuffle(seed=np.random.randint(1000))
dataset = shuffled_dataset.select(range(500))

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for mozilla-foundation/common_voice_17_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_17_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
from transformers import AutoProcessor, AutoModelForCTC

In [13]:
import torch
from datasets import load_dataset
from transformers import AutoProcessor, AutoModelForCTC
import torchaudio
import librosa
import numpy as np
import re
import string
from datasets import load_metric
# Load model and processor
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\]\[\{\}\־]'

FILTER_THRESHOLD = 15

# Load the dataset
def remove_special_characters(batch):
    batch[transcription_key_in_dictionary] = re.sub(chars_to_remove_regex, '', batch[transcription_key_in_dictionary]).lower()
    return batch

# Function to drop samples that contain English letters or digits
def drop_english_samples(dataset):
    def contains_english_or_digits(text):
        english_letters = set(string.ascii_lowercase)
        digits = set(string.digits)
        return any(char in english_letters or char in digits for char in text.lower())
    filtered_dataset = dataset.filter(lambda example: not contains_english_or_digits(example[transcription_key_in_dictionary]))
    return filtered_dataset

# Function to filter out long audio samples
def filter_long_samples(dataset):
    def is_shorter_than_max_duration(example):
        duration_seconds = len(example['audio']['array']) / example['audio']['sampling_rate']
        return duration_seconds <= FILTER_THRESHOLD
    filtered_dataset = dataset.filter(lambda example: is_shorter_than_max_duration(example))
    return filtered_dataset

def remove_hebrew_vowels(batch):
    batch[transcription_key_in_dictionary]= re.sub("[\u05B0-\u05C3\u05C7\u05C4]", "", batch[transcription_key_in_dictionary])
    return batch
# Apply the functions to the dataset
dataset = dataset.map(remove_special_characters)
dataset = drop_english_samples(dataset)
dataset = filter_long_samples(dataset)
dataset = dataset.map(remove_hebrew_vowels)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [14]:
import torch
from tqdm import tqdm
wer = load_metric("wer")
cer = load_metric("cer")
# Setup device

# Assuming the predict function is defined elsewhere and needs to be updated
def predict(audio_tensor, model, processor=None, whisper=False):
    # Ensure the tensor is on the correct device
    # audio_tensor = audio_tensor.to(device)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if not whisper:
        try:
            input_values = model.processor([audio_tensor], return_tensors="pt", sampling_rate=16_000).input_features
        except:
            input_values = model.processor([audio_tensor], return_tensors="pt", sampling_rate=16_000).input_values
        input_values = input_values.to(device)  # Ensure inputs are on the same device as the model
        features = input_values.to(device)

        prediction = model.get_prediction(features)
        return prediction[0]
    else:
        input_features = processor(audio_tensor, sampling_rate=16000, return_tensors="pt")
        input_features = input_features.to(device)
        predicted_ids = model.generate(input_features.input_features, language='he', num_beams=5)
        transcript = processor.batch_decode(predicted_ids, skip_special_tokens=True)
        return transcript[0]

def evaluate_model(dataset, model, processor=None, whisper=False):
    total_wer = 0
    total_cer = 0
    count = 0  # To count successfully processed examples
    with tqdm(total=len(dataset), desc="Evaluating model", unit=" samples") as progress_bar:
        for example in dataset:
            speech_array, sampling_rate = example["audio"]["array"], example["audio"]["sampling_rate"]
            # Convert numpy array to tensor and then move it to the desired device for processing
            # speech_tensor = torch.tensor(speech_array, dtype=torch.float32).to(device)
            # # Explicitly pass the original sampling rate and move tensor to CPU for librosa processing
            # speech_array = speech_tensor.cpu().numpy()
            speech_array = librosa.resample(speech_array, orig_sr=sampling_rate, target_sr=16000)
            # Convert the resampled array back to tensor and move to the correct device
            predicted_text= predict(speech_array, model, processor, whisper=whisper)
            # Compute WER
            try:
                example_wer = wer.compute(references=[example[transcription_key_in_dictionary]], predictions=[predicted_text])
                example_cer = cer.compute(references=[example[transcription_key_in_dictionary]], predictions=[predicted_text])
                total_wer += example_wer
                total_cer += example_cer
                count += 1
            except:
                print("error")
            progress_bar.update(1)

        progress_bar.close()
        if count > 0:
            average_wer = total_wer / count
            average_cer = total_cer / count
        else:
            average_wer = float('inf')  # Indicates an issue if no audio was processed
            average_cer = float('inf')  # Indicates an issue if no audio was processed
        return average_wer, average_cer


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for cer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/cer/cer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandat

In [7]:
from transformers import AutoProcessor, AutoModelForCTC

class ASRModel:
    def __init__(self, model_name=None, model=None, processor=None, feature_extractor=None, tokenizer=None, lm_model=False, device = None):
        self.model_name=model_name
        self.feature_extractor=feature_extractor
        self.processor=processor
        self.tokenizer=tokenizer
        self.lm_model=lm_model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if feature_extractor and tokenizer:
            self.feature_extractor=feature_extractor
            self.tokenizer=tokenizer
            self.processor=AutoProcessor(feature_extractor=feature_extractor, processor=processor)

        elif processor:
            self.processor=processor
        else:
            self.processor = AutoProcessor.from_pretrained(model_name)
        
        print('Getting Model...')
        if lm_model:
            self.model= AutoModelForCTC.from_pretrained(model_name)
        elif model:
           self.model=model 
        else:
            self.model = AutoModelForCTC.from_pretrained(model_name)
        self.model.to(self.device)
            
    def get_prediction(self, inputs, sampling_rate=16000, return_tensors="pt"):
        with torch.no_grad():
            self.logits = self.model(inputs).logits
        if self.lm_model:
            return self.lm_model.batch_decode(self.logits.cpu().numpy()).text
        else:
            predicted_ids = torch.argmax(self.logits, dim=-1)
            return self.processor.batch_decode(predicted_ids)


Wav2Vec2-LM

In [18]:
# processor = AutoProcessor.from_pretrained("imvladikon/wav2vec2-xls-r-300m-hebrew")
# model = AutoModelForCTC.from_pretrained("imvladikon/wav2vec2-xls-r-300m-hebrew")
# model.to(device)
# model.eval()
lm = Wav2Vec2ProcessorWithLM.from_pretrained("/teamspace/studios/this_studio/models/KenLM-Wav2Vec2-imvladikon-300m-ivritai")
wav2vec_he_lm = ASRModel("imvladikon/wav2vec2-xls-r-300m-hebrew", lm_model=lm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading the LM will be faster if you build a binary file.
Reading /teamspace/studios/this_studio/models/KenLM-Wav2Vec2-imvladikon-300m-ivritai/language_model/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Getting Model...


Some weights of the model checkpoint at imvladikon/wav2vec2-xls-r-300m-hebrew were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at imvladikon/wav2vec2-xls-r-300m-hebrew and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probab

In [19]:
average_wer, average_cer = evaluate_model(dataset, wav2vec_he_lm)
print(f"Average WER: {average_wer:.2f}")
print(f"Average CER: {average_cer:.2f}")

Evaluating model: 100%|██████████| 500/500 [17:07<00:00,  2.05s/ samples]

Average WER: 0.28
Average CER: 0.08


Wav2Vec2Bert-LM

In [20]:
# model_path='models/facebook/w2v-bert-2.0-finetuned'
# processor = Wav2Vec2BertProcessor.from_pretrained(model_path, 
#                                             unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
# model = Wav2Vec2BertForCTC.from_pretrained(
#     model_path,
#     pad_token_id=processor.tokenizer.pad_token_id,
#     vocab_size=len(processor.tokenizer),
# )
# model.to(device)
# model.eval()
processor = Wav2Vec2BertProcessor.from_pretrained('models/facebook/w2v-bert-2.0-finetuned', 
                                            unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|"
)

bertLM = Wav2Vec2ProcessorWithLM.from_pretrained("/teamspace/studios/this_studio/models/wav2vec2bert-ivritai")
wav2vec2BertLm = ASRModel(model_name="models/facebook/w2v-bert-2.0-finetuned", processor=processor, lm_model=bertLM)

Loading the LM will be faster if you build a binary file.
Reading /teamspace/studios/this_studio/models/wav2vec2bert-ivritai/language_model/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


Getting Model...


In [21]:
# Run evaluation
average_wer, average_cer = evaluate_model(dataset, wav2vec2BertLm)
print(f"Average WER: {average_wer:.2f}")
print(f"Average CER: {average_cer:.2f}")

Evaluating model: 100%|██████████| 500/500 [19:03<00:00,  2.29s/ samples]

Average WER: 0.23
Average CER: 0.06


Open-AI Whisper

In [8]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration

has_cuda = torch.cuda.is_available()

model_path = 'openai/whisper-large-v2'

model = WhisperForConditionalGeneration.from_pretrained(model_path)
if has_cuda:
    model.to('cuda:0')

processor = WhisperProcessor.from_pretrained(model_path)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
average_wer, average_cer = evaluate_model(dataset, model, processor, whisper=True)
print(f"Average WER: {average_wer:.2f}")
print(f"Average CER: {average_cer:.2f}")

Evaluating model: 100%|██████████| 500/500 [24:57<00:00,  3.00s/ samples]

Average WER: 0.28
Average CER: 0.07


ivrit-ai whisper

In [16]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
model_path = 'ivrit-ai/whisper-large-v2-tuned'
model = WhisperForConditionalGeneration.from_pretrained(model_path)
model=model.to(device)
processor = WhisperProcessor.from_pretrained(model_path)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
average_wer, average_cer = evaluate_model(dataset, model, processor, whisper=True)
print(f"Average WER: {average_wer:.2f}")
print(f"Average CER: {average_cer:.2f}")

Evaluating model: 100%|██████████| 500/500 [25:30<00:00,  3.06s/ samples]

Average WER: 0.27
Average CER: 0.07


In [ ]:
import soundfile as sf

In [15]:
example=dataset[3]
example

{'client_id': '281c3cf84bbed9fbdc383d3b3c89633b2a96b234403bb99a087fbaaacd8623e344c884839cbff5a68e37f88352b3e8139a3cbd3a43ae2e2d7f84af420a08dd2a',
 'path': '/home/zeus/.cache/huggingface/datasets/downloads/extracted/cc1985eee14bdfd38a365480ef760ea8bf84f8b7d1c980d7b50a502369bdb0e5/he_train_0/common_voice_he_39666459.mp3',
 'audio': {'path': '/home/zeus/.cache/huggingface/datasets/downloads/extracted/cc1985eee14bdfd38a365480ef760ea8bf84f8b7d1c980d7b50a502369bdb0e5/he_train_0/common_voice_he_39666459.mp3',
  'array': array([ 6.77626358e-21, -7.45388994e-20, -1.01643954e-19, ...,
          3.68106521e-05,  2.69596840e-05,  1.26023469e-05]),
  'sampling_rate': 48000},
 'sentence': 'היום אתה נותן תעודת חסינות לעבריינים',
 'up_votes': 2,
 'down_votes': 0,
 'age': 'twenties',
 'gender': 'male_masculine',
 'accent': 'Natibe',
 'locale': 'he',
 'segment': '',
 'variant': ''}

In [16]:
sf.write("comparison_test.wav", example['audio']['array'], 32000)

In [17]:
speech_array, sampling_rate = example["audio"]["array"], example["audio"]["sampling_rate"]
speech_array = librosa.resample(speech_array, orig_sr=sampling_rate, target_sr=16000)

In [18]:
from transformers import pipeline
pipe = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-large-v2",
  # chunk_length_s=30,
  device=device,
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [1]:
#More tests to do:
# 1) Test against Whisper ivrit-ai model
# 2) Test against Models without LM

In [19]:

transcription = pipe([speech_array], return_timestamps=True, generate_kwargs={"task": "transcribe", "language": "<|he|>"}, batch_size=1)
example_wer = wer.compute(references=[example[transcription_key_in_dictionary]], predictions=[transcription[0]['text']])
example_cer = cer.compute(references=[example[transcription_key_in_dictionary]], predictions=[transcription[0]['text']])

In [20]:
example_wer

0.16666666666666666

In [21]:
print(transcription[0]['text'])
print(example[transcription_key_in_dictionary])

 היום אתה נותן תעודת חסינות לאבריינים.
היום אתה נותן תעודת חסינות לעבריינים


In [22]:
inputs = processor([speech_array], sampling_rate=16_000).input_values
features = torch.tensor(inputs)

with torch.no_grad():
  logits = model(features).logits

predicted_ids = torch.argmax(logits, dim=-1)
predictions = processor.batch_decode(predicted_ids, skip_special_tokens=True)

AttributeError: 

In [ ]:
predictions

['אבל היא בן צעוד נכנה בעץ לחץ בריתהיא מאוד גדול שמופעל עליו להפסיק את משע כבושים']

In [ ]:
import re

def remove_hebrew_vowels(text):
    return re.sub("[\u05B0-\u05C3\u05C7\u05C4]", "", text)

text_without_vowels = remove_hebrew_vowels(example['sentence'])
print(text_without_vowels)

אחר כך סגרה את הדלת לחדר הסמוך ברעש וניגשה אליה בפסיעות גסות


In [ ]:
wer = load_metric("wer")
cer = load_metric("cer")

/tmp/ipykernel_12055/3798879709.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer = load_metric("wer")
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/datasets/load.py:752: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/datasets/load.py:752: FutureWarning: The repository for cer contains custom code which must be executed to correctly load the m

In [ ]:
cer.compute(references=[example[transcription_key_in_dictionary]], predictions=predictions)

0.1794871794871795

In [ ]:
example[transcription_key_in_dictionary]

'אבל אבן סעוד נכנע בעצם ללחץ בריטי מאוד גדול שמופעל עליו להפסיק את מסע הכיבושים'